# Transformer all in one

In [199]:
%%capture
%load_ext autoreload
%autoreload 2

[autoreload of transformer_all failed: Traceback (most recent call last):
  File "C:\Users\joedi\AppData\Roaming\Python\Python311\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\joedi\AppData\Roaming\Python\Python311\site-packages\IPython\extensions\autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "C:\Users\joedi\AppData\Roaming\Python\Python311\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "C:\Users\joedi\AppData\Roaming\Python\Python311\site-packages\IPython\extensions\autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\joedi\AppData\Roaming\Python\Python311\site-packages\IPython\extensions\autoreload.py", line 397, in update_generic
    update(a, b)
  File "C:\Users\joedi\AppData\Roaming\Python\Python311\site-packages\IPython\

## Imports

In [200]:
import transformer_all
from transformer_all import Config, TrainHandler, DataHandler, Modules, Format

In [201]:
format = Format(style="solarized-light")
print_py = format.print_py
print_obj = format.print_obj
print_src = format.print_src

In [202]:
config=Config()
print_obj(config)

Config(seed=0,
       preview=slice(None, 10, None),
       example_item=1337,
       max_length=512,
       min_freq=5,
       default_token='<unk>',
       special_tokens=['<unk>', '<pad>'],
       batch_size=512,
       lr=0.001,
       num_epochs=20,
       heads=4,
       emb=32,
       device='cuda',
       start_time=1710126444.3235316,
       last_time=0.0)



In [203]:
train_handler = TrainHandler(config=config)
print_src(train_handler.__init__)

    def __init__(self, config: Config):
        self.config = config
        np.random.seed(self.config.seed)
        torch.manual_seed(self.config.seed)
        torch.cuda.manual_seed(self.config.seed)
        torch.backends.cudnn.deterministic = True



In [204]:
data_handler = DataHandler(config=config)
print_src(data_handler.__init__)

  0.141700s   0.000000s |	DataHandler Start
  1.420663s   1.278964s |	DataHandler load_dataset
  1.420663s   0.000000s |	DataHandler get_tokenizer


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

  7.252024s   5.831361s |	DataHandler tokenize_all
 10.035420s   2.783396s |	DataHandler set_vocab


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

 21.062289s  11.026868s |	DataHandler numericalize_all
 21.063289s   0.001000s |	DataHandler format_all
 21.063289s   0.000000s |	DataHandler set_data_loaders
    def __init__(self, config: Config):
        self.config = config
        self.config.elapsed("DataHandler Start")
        self.dataset = datasets.load_dataset("zapsdcn/imdb", cache_dir="../data/classification/")
        self.config.elapsed("DataHandler load_dataset")
        self.train_dataset = self.dataset['train'] 
        self.validation_dataset = self.dataset['validation']
        self.test_dataset = self.dataset['test']
        self.tokenizer = torchtext.data.utils.get_tokenizer("basic_english")
        self.config.elapsed("DataHandler get_tokenizer")
        self.trim = slice(None,config.max_length,None)
        self.tokenize_all()
        self.config.elapsed("DataHandler tokenize_all")
        self.set_vocab()
        self.config.elapsed("DataHandler set_vocab")
        self.numericalize_all()
        self.config.elap

In [205]:
train_handler.load(data_handler=data_handler)
print_src(train_handler.load)

 21.145303s   0.082014s |	TrainHandler start
 21.154303s   0.009000s |	TrainHandler load
    def load(self, data_handler: DataHandler):
        self.config.elapsed("TrainHandler start")
        self.data_handler = data_handler
        self.model = Modules.Transformer(config=self.config, vocab_size=len(self.data_handler.vocab)).to(self.config.device)
        self.opt = torch.optim.Adam(lr=self.config.lr, params=self.model.parameters())
        self.config.elapsed("TrainHandler load")



In [206]:
print_src(Modules.Transformer)

    class Transformer(torch.nn.Module):
        def __init__(self, config: Config, vocab_size: int):
            super().__init__()
            self.config = config
            self.token_embedding = torch.nn.Embedding(embedding_dim=self.config.emb, num_embeddings=vocab_size)
            self.pos_embedding = torch.nn.Embedding(embedding_dim=self.config.emb, num_embeddings=self.config.max_length)
            self.tblock = Modules.TransformerBlock(config=self.config)
            self.toprobs = torch.nn.Linear(self.config.emb, 2)

        def forward(self, x):
            tokens = self.token_embedding(x)
            b, t, e = tokens.shape
            positions = self.pos_embedding(torch.arange(t, device=self.config.device))[None, :, :].expand(b, t, e)
            x = tokens + positions
            x, W = self.tblock(x)
            x = torch.mean(x, dim=1)
            x = self.toprobs(x)
            return F.log_softmax(x, dim=1), W



In [207]:
train_handler.train()

 21.269282s   0.114979s |	train start
 21.283789s   0.014507s |	train batch start
 21.285791s   0.002002s |	train batch ready
 21.286791s   0.001000s |	train batch pred
 21.292791s   0.006000s |	train batch step
 25.037649s   3.744858s |	train batch start
 25.038650s   0.001000s |	train batch ready
 25.039650s   0.001000s |	train batch pred
 25.041650s   0.002000s |	train batch step
 28.777332s   3.735682s |	train batch start
 28.777332s   0.000000s |	train batch ready
 28.778332s   0.001000s |	train batch pred
 28.780839s   0.002507s |	train batch step
 32.449634s   3.668796s |	train batch start
 32.450635s   0.001001s |	train batch ready
 32.451632s   0.000997s |	train batch pred
 32.453632s   0.002000s |	train batch step
 36.076785s   3.623153s |	train batch start
 36.077789s   0.001004s |	train batch ready
 36.078785s   0.000996s |	train batch pred
 36.079785s   0.001000s |	train batch step
 39.711839s   3.632054s |	train batch start
 39.711839s   0.000000s |	train batch ready
 39.

KeyboardInterrupt: 

In [ ]:
train_handler.save()